# Arbor Foods Trading Co.


**Note:** This is a SQLite-powered demo version of the Arbor Foods project.  
Some PostgreSQL-specific syntax has been modified for compatibility.


## Load SQL extension for IPython and connect to database

The following lines of code will provide the ability to write SQL queries in the Jupyter Notebook:


In [1]:
# This command loads the sql extension for IPython
%load_ext sql

# This command establishes a connection to the Arbor Foods database
%sql sqlite:///arborfoods.db

The below removes any views that were created during a previous run of this notebook.


In [2]:
%%sql
DROP VIEW customer_orders;
DROP VIEW detailed_orders;
DROP VIEW employee_orders;

 * sqlite:///arborfoods.db
Done.
Done.
Done.


[]

## Getting to Know the Data


### List all tables and views

To obtain a list of all tables and views in the SQLite database, the built-in `sqlite_master` table can be queried:


In [3]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ('table', 'view')
   AND name NOT LIKE 'sqlite_%';

 * sqlite:///arborfoods.db
Done.


name,type
suppliers,table
customers,table
territories,table
region,table
us_states,table
customer_demographics,table
customer_customer_demo,table
employees,table
employee_territories,table
orders,table


## Create Views

I'll be creating views that will help with the rest of the project.


### A view with order and customer information

First, combining the `orders` and `customers` tables to get more detailed information about each order:


In [4]:
%%sql
CREATE VIEW customer_orders AS
SELECT o.order_id,
       c.company_name,
       c.customer_id, 
       c.contact_name,
       o.order_date
  FROM orders AS o
  JOIN customers AS c
    ON c.customer_id = o.customer_id;

 * sqlite:///arborfoods.db
Done.


[]

The first 10 rows of the `customer_orders` view:


In [5]:
%%sql
SELECT *
  FROM customer_orders
 LIMIT 10;

 * sqlite:///arborfoods.db
Done.


order_id,company_name,customer_id,contact_name,order_date
10248,Vins et alcools Chevalier,VINET,Paul Henriot,1996-07-04
10249,Toms Spezialitäten,TOMSP,Karin Josephs,1996-07-05
10250,Hanari Carnes,HANAR,Mario Pontes,1996-07-08
10251,Victuailles en stock,VICTE,Mary Saveley,1996-07-08
10252,Suprêmes délices,SUPRD,Pascale Cartrain,1996-07-09
10253,Hanari Carnes,HANAR,Mario Pontes,1996-07-10
10254,Chop-suey Chinese,CHOPS,Yang Wang,1996-07-11
10255,Richter Supermarkt,RICSU,Michael Holz,1996-07-12
10256,Wellington Importadora,WELLI,Paula Parente,1996-07-15
10257,HILARION-Abastos,HILAA,Carlos Hernández,1996-07-16


### A view with detailed order information


The next view will combine the `order_details`, `products`, and `orders` tables to get detailed order information.


In [6]:
%%sql
CREATE VIEW detailed_orders AS 
SELECT o.order_id,
       o.order_date,
       p.product_name,
       p.product_id,
       od.quantity,
       od.unit_price, 
       od.discount
  FROM orders AS o
  JOIN order_details AS od
    ON o.order_id = od.order_id
  JOIN products AS p
    ON od.product_id = p.product_id;

 * sqlite:///arborfoods.db
Done.


[]

The first 10 rows of the newly created `detailed_orders` view:


In [7]:
%%sql
SELECT *
  FROM detailed_orders
 LIMIT 10;

 * sqlite:///arborfoods.db
Done.


order_id,order_date,product_name,product_id,quantity,unit_price,discount
10248,1996-07-04,Queso Cabrales,11,12,14.0,0.0
10248,1996-07-04,Singaporean Hokkien Fried Mee,42,10,9.8,0.0
10248,1996-07-04,Mozzarella di Giovanni,72,5,34.8,0.0
10249,1996-07-05,Tofu,14,9,18.6,0.0
10249,1996-07-05,Manjimup Dried Apples,51,40,42.4,0.0
10250,1996-07-08,Jack's New England Clam Chowder,41,10,7.7,0.0
10250,1996-07-08,Manjimup Dried Apples,51,35,42.4,0.15
10250,1996-07-08,Louisiana Fiery Hot Pepper Sauce,65,15,16.8,0.15
10251,1996-07-08,Gustaf's Knäckebröd,22,6,16.8,0.05
10251,1996-07-08,Ravioli Angelo,57,15,15.6,0.05


### A view with employee and order information


Combining the `employees` and `orders` tables will provide information on which employee was responsible for each order.


In [8]:
%%sql
CREATE VIEW employee_orders AS
SELECT e.employee_id,
       e.first_name || ' ' || e.last_name AS employee_name,
       o.order_id, 
       o.order_date
  FROM employees AS e
  JOIN orders AS o
    ON e.employee_id = o.employee_id;

 * sqlite:///arborfoods.db
Done.


[]

The first 10 rows of the `employee_orders` view:


In [9]:
%%sql
SELECT *
  FROM employee_orders
 LIMIT 10;

 * sqlite:///arborfoods.db
Done.


employee_id,employee_name,order_id,order_date
1,Nancy Davolio,10258,1996-07-17
1,Nancy Davolio,10270,1996-08-01
1,Nancy Davolio,10275,1996-08-07
1,Nancy Davolio,10285,1996-08-20
1,Nancy Davolio,10292,1996-08-28
1,Nancy Davolio,10293,1996-08-29
1,Nancy Davolio,10304,1996-09-12
1,Nancy Davolio,10306,1996-09-16
1,Nancy Davolio,10311,1996-09-20
1,Nancy Davolio,10314,1996-09-25


## Ranking Employee Sales Performance


Ranking employees based on their total sales amount will allow management to recognize and reward top-performing employees, foster a culture of excellence within the organization, as well as identify employees who might be struggling so management can offer the necessary training or resources to help them improve.


The following creates a Common Table Expression (CTE) that calculates the total sales for each employee using the employee_orders view and order_details table. Then, the next command ranks each employee based on their total sales:


In [10]:
%%sql
WITH total_sales_by_employee AS(
  SELECT e.employee_id, 
         e.employee_name,
         ROUND(SUM(od.quantity * od.unit_price * (1-od.discount)), 2) AS total_sales
    FROM employee_orders AS e
    JOIN order_details AS od
      ON e.order_id = od.order_id
   GROUP BY e.employee_id, e.employee_name
)

SELECT employee_id AS "Emp ID", 
       employee_name AS "Emp Name",
       total_sales AS "Total Sales",
       RANK() OVER(ORDER BY total_sales DESC) AS "Sales Rank"
  FROM total_sales_by_employee;

 * sqlite:///arborfoods.db
Done.


Emp ID,Emp Name,Total Sales,Sales Rank
4,Margaret Peacock,232890.85,1
3,Janet Leverling,202812.84,2
1,Nancy Davolio,192107.6,3
2,Andrew Fuller,166537.76,4
8,Laura Callahan,126862.28,5
7,Robert King,124568.24,6
9,Anne Dodsworth,77308.07,7
6,Michael Suyama,73913.13,8
5,Steven Buchanan,68792.28,9


Based on the above table, `Margaret Peacock` is the top rank employee in regards to total sales with $232,890.85 in total sales.

Conversely, the table also shows `Steven Buchanan` has having the least amount of sales among all the employees with a total sales of $68,792.28.


## Running Total of Monthly Sales


Creating a running total of sales by month will provide a more macro-level perspective around the company's overall sales performance over time, which will help management identify trends that might shape the company's future strategies.


For this analysis task, the `orders` and `order_details` tables will be needed. Luckily, a view combining these tables was created earlier, the `detailed_orders` view.

The following query creates a CTE called `monthly_sales` calculates the total sales per month using the `quantity`, `unit_price`, `discount`, and `order_date` columns from the `detailed_orders` view.

A second query uses the CTE to calculate a running total of total sales per month.


In [11]:
%%sql
WITH monthly_sales AS(
    SELECT strftime('%Y-%m', order_date) AS month,
           Round(SUM(unit_price * quantity * (1 - discount)), 2) AS total_sales
      FROM detailed_orders
     GROUP BY strftime('%Y-%m', order_date)
)

SELECT month AS "Month",
       SUM(total_sales) OVER(ORDER BY month) AS "Running Total"
  FROM monthly_sales
 ORDER BY month;


 * sqlite:///arborfoods.db
Done.


Month,Running Total
1996-07,27861.9
1996-08,53347.18
1996-09,79728.58
1996-10,117244.3
1996-11,162844.34
1996-12,208083.97
1997-01,269342.04
1997-02,307825.68
1997-03,346372.9
1997-04,399405.85


## Month-Over-Month Sales Growth


Analyzing the month-over-month sales growth rate will provide a better understanding of the rate at which sales are increasing or decreasing, and will help the management team to identify significant trends.


The following query will compare each month's sales with the previous month's, then calculate the percentage change in sales.


In [12]:
%%sql
WITH monthly_sales AS(
    SELECT strftime('%m', order_date) AS month,
           strftime('%Y', order_date) AS year,
           ROUND(SUM(unit_price * quantity * (1 - discount)), 2) AS total_sales
      FROM detailed_orders
     GROUP BY year, month
),

previous_sales AS(
    SELECT month,
           year,
           total_sales,
           LAG(total_sales) OVER(ORDER BY year, month) AS previous_month_sales
      FROM monthly_sales
)

SELECT year AS "Year",
       month AS "Month",
       total_sales AS "Monthly Sales",
       previous_month_sales AS "Previous Month Sales",
       ROUND((total_sales / previous_month_sales - 1) * 100, 2) AS "Sales Growth Rate"
  FROM previous_sales;

 * sqlite:///arborfoods.db
Done.


Year,Month,Monthly Sales,Previous Month Sales,Sales Growth Rate
1996,07,27861.9,None,None
1996,08,25485.28,27861.9,-8.53
1996,09,26381.4,25485.28,3.52
1996,10,37515.72,26381.4,42.21
1996,11,45600.04,37515.72,21.55
1996,12,45239.63,45600.04,-0.79
1997,01,61258.07,45239.63,35.41
1997,02,38483.64,61258.07,-37.18
1997,03,38547.22,38483.64,0.17
1997,04,53032.95,38547.22,37.58


## Identifying High-Value Customers


Offering targeted promotions and special offers to customers with above-average order values could lead to an increase in sales, improved customer retention, and attract new customers.


The following query joins the `customer_orders` view with the `order_details` table into a CTE called `customer_sales` which calculates the sale amount for each order.

Another CTE called `labeld_sales` uses the `customer_sales` CTE to calculate the average sale amount per customer.

A third and final CTE called `above_avg_counts` counts the number of above average purchases per customer.

The final query uses the `above_avg_counts` CTE to rank all of the customers based on how many above-average orders they have made. The final output has been truncated to only show the top 10 companies.


In [13]:
%%sql
WITH customer_sales AS(
    SELECT c.customer_id,
           (od.quantity * od.unit_price * (1 - od.discount)) AS sale_amount
      FROM customer_orders AS c
      JOIN order_details AS od
        ON c.order_id = od.order_id
),

labled_sales AS (
    SELECT customer_id,
           sale_amount,
           AVG(sale_amount) OVER(PARTITION BY customer_id) AS avg_sale
      FROM customer_sales
),

above_avg_counts AS (
    SELECT customer_id,
           COUNT(*) FILTER(WHERE sale_amount > avg_sale) AS above_avg_count
      FROM labled_sales
     GROUP BY customer_id
)

SELECT customer_id AS "Customer ID",
       above_avg_count "Above-Average Orders",
       RANK() OVER(ORDER BY above_avg_count DESC) AS "Rank"
  FROM above_avg_counts
 LIMIT 10;

 * sqlite:///arborfoods.db
Done.


Customer ID,Above-Average Orders,Rank
SAVEA,40,1
ERNSH,34,2
QUICK,28,3
BONAP,19,4
FRANK,19,4
RATTC,17,6
HILAA,16,7
BOTTM,15,8
HUNGO,14,9
KOENE,14,9


From the output above we can see the top 10 companies with the highest amount of above-average order values. The customer `SAVEA` is at the top of the list with 40 above-average orders.


## Percentage of Sales for Each Category


By knowing the percentage of total sales for each product category, management will have better insights into which categories drive most of the company's sales. This will help guide decisions about inventory and marketing strategies.


The first below creates a CTE called `sales_per_category` which calculates the sale amount for each product category.

The second query outputs the IDs, names, and percentage of sales for each category.


In [14]:
%%sql
WITH sales_per_category AS (
    SELECT c.category_id,
           c.category_name,
           SUM(od.unit_price * od.quantity * (1 - od.discount)) AS sales_amount
      FROM categories AS c
      JOIN products AS p
        ON c.category_id = p.category_id
      JOIN order_details AS od
        ON p.product_id = od.product_id
    GROUP BY c.category_id
)

SELECT category_id AS "Category ID",
       category_name AS "Category Name",
       ROUND(((sales_amount / SUM(sales_amount) OVER(ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) * 100), 1) AS "% Total Sales"
  FROM sales_per_category
 ORDER BY "% Total Sales" DESC;

 * sqlite:///arborfoods.db
Done.


Category ID,Category Name,% Total Sales
1,Beverages,21.2
4,Dairy Products,18.5
3,Confections,13.2
6,Meat/Poultry,12.9
8,Seafood,10.4
2,Condiments,8.4
7,Produce,7.9
5,Grains/Cereals,7.6


From the above table, the `Beverages` category makes up the most of the company's total sales at about 21.2%. `Grains/Cereals` and `Produce` make up the least of the company's sales making up about 7.6% and 7.9% of all sales, respectively.


## Top Products Per Category


The final objective will be to provide management with a list of the top three items sold in each product category, which will allow them to identify top performers and to ensure these products are kept in stock.


The first query below creates a CTE called `product_sales` that calculates the total sales for each product.

The second query outputs the top three products from each category based on their total sales.


In [15]:
%%sql
WITH product_sales AS (
    SELECT p.product_name,
           p.product_id,
           p.category_id,
           ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount)), 2) AS total_sales
      FROM products AS p
      JOIN order_details AS od
        ON p.product_id = od.product_id
     GROUP BY p.category_id, p.product_id
)

SELECT category_id AS "Category ID",
       product_id AS "Product ID",
       product_name AS "Product Name",
       total_sales AS "Total Sales"
  FROM (SELECT category_id,
               product_id,
               product_name,
               total_sales,
               ROW_NUMBER() OVER(PARTITION BY category_id ORDER BY total_sales DESC) AS row_num
          FROM product_sales
       ) AS tmp
 WHERE row_num <= 3;

 * sqlite:///arborfoods.db
Done.


Category ID,Product ID,Product Name,Total Sales
1,38,Côte de Blaye,141396.73
1,43,Ipoh Coffee,23526.7
1,2,Chang,16355.96
2,63,Vegie-spread,16701.1
2,61,Sirop d'érable,14352.6
2,65,Louisiana Fiery Hot Pepper Sauce,13869.89
3,62,Tarte au sucre,47234.97
3,20,Sir Rodney's Marmalade,22563.36
3,26,Gumbär Gummibärchen,19849.14
4,59,Raclette Courdavault,71155.7
